In [3]:
#WEIGHTLIFTING RANKINGS BUILDER AND TEAM SELECTOR (V1)
#Includes current rankings updated on Dec 22nd

#Last modified on Dec 22, 2024 by M. H. Kent 

In [4]:
#Import needed packages 
import pandas as pd
import numpy as np 
import glob 
import os
import math
from datetime import datetime


In [5]:
# 1) Functions for Sinclair Coefficent and classifications by age and preformance

#Sets up the sinclair constant sheet for use 
def sinclair_constant_setup(path, file_name):
    path_to_sheet = path + "/" + file_name
    sinclair_sheet_up = pd.read_csv(path_to_sheet)
    sinclair_sheet = sinclair_sheet_up.drop(index=2)
    return sinclair_sheet

#Defines the sinclair coefficent formula
def sinclair_coef(x, A, b):
    div_const = float(float(x)/float(b))
    big_X = math.log10(div_const)
    if x <= b:
        return 10**(A*(big_X**2))
    if x > b: 
        return 1

#Defines the sinclair age coefficent modeification for masters athletes (Will need to check this) 
def sinclair_masters_mod(Ag, Age): 
    return 1 + ((Ag/100)*(Age - 30))

#Calculates the sinclair total from the coefficent
def sinclair_total(total, x, A, b):
    return total * sinclair_coef(x, A, b)

#Calculates the masters Sinclair total 
def masters_sinclair_total(total, x, A, b, Ag, Age):
    return total * sinclair_coef(x, A, b) * sinclair_masters_mod(Ag, Age)

#Rounds each float in a list to the nearest whole integer 
def list_rounder(num_list:list):
    return [round(num) for num in num_list]

#Read and set up classification tables 
def classification_setup(path, file_name):
    path_to_sheet = path + "/" + file_name
    mens_class_base = pd.read_excel(path_to_sheet, skiprows=3, nrows=11, usecols=lambda column: column != 'Unnamed: 0')
    mens_class = ((mens_class_base.drop(index=0)).reset_index(drop=True)).round(0)
    mens_class['Novice'] = list_rounder(list(mens_class.loc[:,'Novice'])) #Rounds the one row that did not automatically round for some reason
    mens_class['Category'] = mens_class['Category'].str.replace('kg', '', regex=False)
    womans_class_base = pd.read_excel(path_to_sheet, skiprows=16, nrows=11, usecols=lambda column: column != 'Unnamed: 0')
    womans_class = ((womans_class_base.drop(index=0)).reset_index(drop=True)).round(0)
    womans_class['Novice'] = list_rounder(list(womans_class.loc[:,'Novice'])) #Rounds the one row that did not automatically round for some reason
    womans_class['Category'] = womans_class['Category'].str.replace('kg', '', regex=False)
    return mens_class, womans_class

#Sets up the age constant dataframe so it can be used
def age_const_setup(path, file_name):
    path_to_sheet = path + "/" + file_name
    class_base = pd.read_csv(path_to_sheet)
    class_base[["Age_low", "Age_high"]] = (class_base['Age_group'].str.split('-', expand=True)).reset_index(drop=True)
    class_base["Age_low"] = class_base["Age_low"].str.replace('+', '', regex=False)
    return class_base

#Identifies the proper coefficents needed for the sinclare formula for age (Will need to check this)
def age_constant_selector(mast_const_df, min_age_colname, max_age_colname, const_colname, ath_age):
    for i in range((mast_const_df.shape[0])): 
        age_mins = float(mast_const_df.loc[i, min_age_colname])
        age_max = float(mast_const_df.loc[i, max_age_colname])
        if age_max == None: 
            return mast_const_df.loc[i, const_colname]
        elif age_mins <= ath_age <= age_max: 
            return mast_const_df.loc[i, const_colname]

#Fills in missing columns so dataframes can be stacked
def column_filler(df1, df2):
    df1_rownum = df1.shape[0]
    df2_rownum = df2.shape[0]
    df1_cols = list(df1.columns)
    df2_cols = list(df2.columns)
    for col in df1_cols:
        if col not in df2_cols:
            filler = []
            for i in range(df2_rownum):
                filler.append('NaN')
            df2[col] = filler
    return df1, df2


In [6]:
# 2) Functions for building a master sheet from update sheet and completeing various computations for each results

#Computing competition ages and adding to the sheet
def comp_age_adder(upd_sheet, upd_rownum):
    ages = []
    for i in range(upd_rownum):
        birth_year = float(upd_sheet.loc[i, 'YOB'])
        comp_date = upd_sheet.loc[i, 'Date'].year
        if pd.isna(comp_date) or pd.isnull(comp_date):
            ages.append('NaN')
        else:
            ages.append(comp_date - birth_year)
    upd_sheet['Comp_Age'] = ages
    upd_sheet = upd_sheet.reset_index(drop=True)
    return upd_sheet

#Computing current ages and adding to the sheet
def curr_age_adder(upd_sheet, upd_rownum):
    ages = []
    for i in range(upd_rownum):
        birth_year = float(upd_sheet.loc[i, 'YOB'])
        comp_date = datetime.now().year
        if pd.isna(comp_date) or pd.isnull(comp_date):
            ages.append('NaN')
        else:
            ages.append(comp_date - birth_year)
    upd_sheet['Current_Age'] = ages
    upd_sheet = upd_sheet.reset_index(drop=True)
    return upd_sheet

#Adding age group catagories
def catagory_adder(upd_sheet, upd_rownum, age_col): 
    catagories = []
    for i in range(upd_rownum): 
        age_in_q =  float(upd_sheet.loc[i, age_col]) #Makes sure it is a number if possible
        if age_in_q < 17: 
            catagories.append('Yth')
        elif age_in_q < 21:
            catagories.append('Jr')
        elif age_in_q < 35:
            catagories.append('Sr')
        elif isinstance(age_in_q , (int, float)): #If it is a number greater then 35
            catagories.append('Mstr')
        else: 
            catagories.append('NaN')
    cat_colname = 'Competition_Group_' + age_col
    upd_sheet[cat_colname] = catagories
    upd_sheet = upd_sheet.reset_index(drop=True)
    return upd_sheet

#Adding Sinclair coeffients
def sinclair_adder(upd_sheet, upd_rownum, age_col, sinclair_table, masters_men_table, masters_woman_table):
    BW_sinclair = []
    WC_sinclair = []
    Mst_sinclair = []
    for i in range(upd_rownum): 
        ath_age = float(upd_sheet.loc[i, age_col])
        ath_total = float(upd_sheet.loc[i, 'Total'])
        ath_sex = upd_sheet.loc[i, 'Sex']
        ath_BW = upd_sheet.loc[i, 'BW']
        ath_wtcl = upd_sheet.loc[i, 'Weight_Class']
        if '+' in ath_wtcl: #Set up so we can use appropriate sinclair for heavyweight
            sin = 1
        else:
            sin = 0
            ath_wtcl = float(ath_wtcl)
        if ath_sex == 'M': 
            BW_sinclair.append(sinclair_total(ath_total, ath_BW, float(sinclair_table.loc[0, 'Men']), float(sinclair_table.loc[1, 'Men'])))
            #Append weight class sinclair
            if sin == 0:
                WC_sinclair.append(sinclair_total(ath_total, float(ath_wtcl), float(sinclair_table.loc[0, 'Men']), float(sinclair_table.loc[1, 'Men'])))
            elif sin == 1:
                WC_sinclair.append(ath_total)
            else:
                WC_sinclair.append('NaN')
            #Append masters sinclair if nessisary
            if ath_age >= 35:
                #5
                age_const_val_m = age_constant_selector(masters_men_table, 'Age_low', 'Age_high', 'Age_coefficient', ath_age)
                Mst_sinclair.append(masters_sinclair_total(ath_total, ath_BW, float(sinclair_table.loc[0, 'Men']), float(sinclair_table.loc[1, 'Men']), age_const_val_m, ath_age))
            else: 
                Mst_sinclair.append('NaN')
        elif ath_sex == 'F':
            BW_sinclair.append(sinclair_total(ath_total, ath_BW, float(sinclair_table.loc[0, 'Woman']), float(sinclair_table.loc[1, 'Woman'])))
            if sin == 0:
                WC_sinclair.append(sinclair_total(ath_total, ath_wtcl, float(sinclair_table.loc[0, 'Woman']), float(sinclair_table.loc[1, 'Woman'])))
            elif sin == 1:
                WC_sinclair.append(ath_total)
            else:
                WC_sinclair.append('NaN')
            if ath_age >= 35:
                age_const_val_w = age_constant_selector(masters_woman_table, 'Age_low', 'Age_high', 'Age_coefficient', ath_age)
                Mst_sinclair.append(masters_sinclair_total(ath_total, ath_BW, float(sinclair_table.loc[0, 'Woman']), float(sinclair_table.loc[1, 'Woman']), age_const_val_w, ath_age))
            else: 
                Mst_sinclair.append('NaN')       
        else: 
            BW_sinclair.append('NaN')
            WC_sinclair.append('NaN')
            Mst_sinclair.append('NaN')
    upd_sheet['BW_Sinclair'] = BW_sinclair
    upd_sheet['WC_Sinclair'] = WC_sinclair
    upd_sheet['Masters_Sinclair'] = Mst_sinclair
    upd_sheet = upd_sheet.reset_index(drop=True)
    return upd_sheet

#Adds the atheletes class to the dataframe
def class_adder(upd_sheet, upd_rownum, mens_index, womans_index, age_col):
    classes = []
    jr_cats = ['Jr. Nats ', 'Next Gen. Elite']
    sr_comp_groups = ['Sr', 'Mstr']
    cols_to_remove = ['Category', 'Marker']
    for i in range(upd_rownum):
        class_l1 = len(classes)
        ath_age = float(upd_sheet.loc[i, age_col])
        ath_total = float(upd_sheet.loc[i, 'Total'])
        ath_sex = upd_sheet.loc[i, 'Sex']
        ath_BW = upd_sheet.loc[i, 'BW']
        ath_wtcl = upd_sheet.loc[i, 'Weight_Class']
        comp_group_colname = 'Competition_Group_' + age_col
        ath_comp_group = upd_sheet.loc[i, comp_group_colname]
        if math.isnan(ath_total):
            classes.append('NaN')
        else:    
            if ath_sex == 'M':
                Msheet_to_det = (mens_index[mens_index['Category'] == ath_wtcl])
                Msheet_to_det = Msheet_to_det.drop(['Category'], axis=1) #Dont need this any more after weightclass selected
                pref_class = list(Msheet_to_det.columns)
                if ath_comp_group in sr_comp_groups:
                    Msheet_to_det =  Msheet_to_det.drop(['Jr. Nats ', 'Next Gen. Elite'], axis=1)
                    pref_class = list(Msheet_to_det.columns)
                Msheet_to_det = (Msheet_to_det[pref_class]).reset_index(drop=True)
                if Msheet_to_det.shape[0] == 0:
                    classes.append('NaN')
                else:
                    for j in range(len(pref_class)-1):
                        classif_b = list((Msheet_to_det.columns))[j]
                        classif_t = list((Msheet_to_det.columns))[j+1]
                        val_bot = float(Msheet_to_det.loc[0, classif_b])
                        val_top = float(Msheet_to_det.loc[0, classif_t])
                        if (classif_b == 'Novice') & (val_bot > ath_total):
                            classes.append(classif_b)
                            break
                        elif (classif_t == 'Elite') & (val_top < ath_total):
                            classes.append(classif_t)
                            break
                        elif val_bot <= ath_total < val_top: 
                            classes.append(classif_b)
                            break 
            elif ath_sex == 'F': 
                Wsheet_to_det = (womans_index[womans_index['Category'] == ath_wtcl])
                Wsheet_to_det = Wsheet_to_det.drop(['Category'], axis=1) 
                pref_class = list(Wsheet_to_det.columns)
                if ath_comp_group in sr_comp_groups:
                    Wsheet_to_det =  Wsheet_to_det.drop(['Jr. Nats ', 'Next Gen. Elite'], axis=1)
                    pref_class = list(Wsheet_to_det.columns)
                Wsheet_to_det = (Wsheet_to_det[pref_class]).reset_index(drop=True)
                if Wsheet_to_det.shape[0] == 0:
                    classes.append('NaN')
                else:
                    for j in range(len(pref_class)-1):
                        classif_b = list((Wsheet_to_det.columns))[j]
                        classif_t = list((Wsheet_to_det.columns))[j+1]
                        val_bot = float(Wsheet_to_det.loc[0, classif_b])
                        val_top = float(Wsheet_to_det.loc[0, classif_t])
                        if (classif_b == 'Novice') & (val_bot > ath_total):
                            classes.append(classif_b)
                            break
                        elif (classif_t == 'Elite') & (val_top < ath_total):
                            classes.append(classif_t)
                            break
                        elif val_bot <= ath_total < val_top: 
                            classes.append(classif_b)
                            break     
    upd_sheet['Class'] = classes
    upd_sheet = upd_sheet.reset_index(drop=True)
    return upd_sheet

#Generates a master sheet from the raw data in the update sheet
def master_sheet_updater(master_path, master_file_name, update_path, update_file_name, mens_index, womans_index, sinclair_table, masters_men_table, masters_woman_table):
    #Read master sheet (make sure it is a .csv file)
    #mst_sheet_path = master_path + "/" + master_file_name
    #old_mst_sheet = pd.DataFrame(pd.read_csv(mst_sheet_path)).reset_index(drop=True)
    master_colnames = ["Last_Name",	"First_Name", "Sex", "Weight_Class", "BW", "Club", "YOB", "Date", "Competition", "Sn_1", "Sn_2", "Sn_3", "Sn_Max", "C&J_1", "C&J_2", "C&J_3", "C&J_max", "Total", "Comp_Age", "Current_Age", "Competition_Group_Current_Age", "Competition_Group_Comp_Age",	"BW_Sinclair", "WC_Sinclair", "Masters_Sinclair", "Class"]
    old_mst_sheet = pd.DataFrame(columns = master_colnames)
    
    #Upload the update sheet sheet
    upd_sheet_path = update_path + "/" + update_file_name
    upd_sheet = pd.DataFrame(pd.read_csv(upd_sheet_path)).reset_index(drop=True)
    upd_sheet['Date'] = pd.to_datetime(upd_sheet['Date'])#, errors='coerce') #Making sure all dates are in datetime format
    #Set some nessisary variables
    upd_rownum = upd_sheet.shape[0]
    
    #Computing current ages
    upd_sheet = comp_age_adder(upd_sheet, upd_rownum)
    
    #Computing compitition ages
    upd_sheet = curr_age_adder(upd_sheet, upd_rownum)
    
    #Computing current age catagory 
    upd_sheet = catagory_adder(upd_sheet, upd_rownum, 'Current_Age')
    
    #Computing compitition
    upd_sheet = catagory_adder(upd_sheet, upd_rownum, 'Comp_Age')
    
    #Computing Sinclair coefficents
    upd_sheet = sinclair_adder(upd_sheet, upd_rownum, 'Comp_Age', sinclair_table, masters_men_table, masters_woman_table)
    
    #Computing Class
    upd_sheet = class_adder(upd_sheet, upd_rownum, mens_index, womans_index, 'Comp_Age')

    #Fills in the blank columns so they match and stacks them to form the updated frame
    old_master_frame, mod_update_frame = column_filler(old_mst_sheet, upd_sheet) 
    updated_master_frame = pd.concat([mod_update_frame, old_master_frame], axis=0).reset_index(drop=True)
    updated_master_frame = updated_master_frame.drop_duplicates()
                        
    #Export the sheet
    current_date = datetime.now()
    date_string = current_date.strftime("%Y-%m-%d")
    output_path = master_path + "/Master_sheet_" + date_string + ".csv"
    #output_path = master_path + "/" + master_file_name
    updated_master_frame.to_csv(output_path, index=False)
    
    #return upd_sheet
    return updated_master_frame



In [7]:
# 3) Functions for computing male and female rankings ina specified date range

#Builds a dataframe that has highest entery for a person by weightclass 
def duplicate_remover(df, firstname_colname, lastname_colname, wtclass_colname, rank_col): 
    cleaned_df = pd.DataFrame(columns = df.columns)
    for i in range(df.shape[0]):
        df_row = (df.iloc[i].to_frame().T).reset_index(drop=True)
        fn_in_col = str(df_row.loc[0, firstname_colname]) in cleaned_df[firstname_colname].values
        ln_in_col = str(df_row.loc[0, lastname_colname]) in cleaned_df[lastname_colname].values
        wc_in_col = str(df_row.loc[0, wtclass_colname]) in cleaned_df[wtclass_colname].values
        if fn_in_col & ln_in_col &  wc_in_col:
            row_in_q = cleaned_df[(cleaned_df[firstname_colname] == df_row.loc[0, firstname_colname]) & (cleaned_df[lastname_colname] == df_row.loc[0, lastname_colname])]
            row_ind =  row_in_q.index[0]
            comp_val = row_in_q.loc[row_ind, rank_col]
            comp_val2 = df_row.loc[0, rank_col] 
            if comp_val <= comp_val2:
                #cleaned_df.loc[row_ind] = df_row
                cleaned_df = pd.concat([cleaned_df, df_row], ignore_index=True)
                cleaned_df = cleaned_df.reset_index(drop=True)    
        else:
            cleaned_df = pd.concat([cleaned_df, df_row], ignore_index=True).reset_index(drop=True)       
    return cleaned_df 

#Adds the rank column to the dataframe 
def rank_adder(df): 
    rank_col = []
    for i in range(1, df.shape[0] + 1):
        rank_col.append(i)
    df.insert(0, "Rank", rank_col)
    return df

#Builds the ranking dataframe 
def all_ranker(master_df, start_date, end_date, firstname_colname, lastname_colname, wtclass_colname, rank_col, nonrank_col_to_inc, export_path, export): 
    inc_cols = nonrank_col_to_inc + [rank_col]
    current_date = datetime.now()
    date_string = current_date.strftime("%Y-%m-%d")
    master_df["Date"] =  pd.to_datetime(master_df["Date"])
    master_df = master_df.reset_index(drop=True) 
    #Sort and organize male rankings dataframe
    male_df_unsort = master_df[(master_df["Sex"] == "M") & (master_df["Date"] >= pd.to_datetime(start_date)) & (master_df["Date"] <= pd.to_datetime(end_date))]
    male_df_sorted = male_df_unsort.sort_values(by=rank_col, ascending=False)
    male_dfa = male_df_sorted[inc_cols].dropna(subset=[rank_col])
    male_df = duplicate_remover(male_dfa, firstname_colname, lastname_colname, wtclass_colname, rank_col)
    male_df_fin = (rank_adder(pd.DataFrame(male_df)).round(0)).reset_index(drop=True)
    male_exp_path = export_path + "/" + "AllMensRankings_" + date_string + ".csv"
    if export == True:
        male_df_fin.to_csv(male_exp_path, index=False)
    #Sort and organize female rankings dataframe
    female_df_unsort = master_df[(master_df["Sex"] == "F") & (master_df["Date"] >= pd.to_datetime(start_date)) & (master_df["Date"] <= pd.to_datetime(end_date))]
    female_df_sorted = female_df_unsort.sort_values(by=rank_col, ascending=False)
    female_dfa = female_df_sorted[inc_cols].dropna(subset=[rank_col])
    female_df = duplicate_remover(female_dfa, firstname_colname, lastname_colname, wtclass_colname, rank_col)
    female_df_fin = (rank_adder(pd.DataFrame(female_df)).round(0)).reset_index(drop=True)
    female_exp_path = export_path + "/" + "AllWomanRankings_" + date_string + ".csv"
    if export == True:
        female_df_fin.to_csv(female_exp_path, index=False)
    return male_df_fin, female_df_fin

#Breaks up a rankings sheet into groups 
def rankings_seperator(df, age_group_colname, age_groups): 
    df_split = df[df[age_group_colname].isin(age_groups)]
    df_split = df_split.reset_index(drop=True)
    df_split = df_split.drop('Rank', axis=1)
    df_split = rank_adder(df_split)
    df_split = df_split.drop(age_group_colname, axis=1)
    return df_split

#A function for simple exporting of rankings
def simple_exporter(df, export_path, df_name):
    df_export = export_path + "/" + df_name + ".csv"
    df.to_csv(df_export, index=False)  



In [8]:
# 4) Functions for internal team selections 

#Add the columns needed for manual organization
def org_col_adder(df, org_cols):
    for col in org_cols:
        df[col] = None
    return df

#Groups the dataframe by weightclass
def wc_grouper(df, weight_class_colname, rank_include):
    wtclsss = df[weight_class_colname].unique()
    df2 = pd.DataFrame() #Dont need to define as we are not pulling anything from it
    for wtclass in wtclsss: 
        df_class = df[df[weight_class_colname] == wtclass]
        if rank_include: 
            df_class = rank_adder(df_class)
        df2 = pd.concat([df2, df_class], ignore_index=True).reset_index(drop=True)
    return df2

#Organizes dataframe for team selection
def team_selector_internal(master_df, start_date, end_date, firstname_colname, lastname_colname, rank_col, age_groups, age_group_col, weight_class_colname, nonrank_col_to_inc, org_cols, export_path, export):
    #Getting the ranks of each athlete 
    mens_rankings, womans_rankings = all_ranker(master_df, start_date, end_date, firstname_colname, lastname_colname, weight_class_colname, rank_col, nonrank_col_to_inc, export_path, False)
    mens_rankings = mens_rankings.drop('Rank', axis=1)#, inplace=True)
    womans_rankings = womans_rankings.drop('Rank', axis=1)#, inplace=True)
    
    #Organizing men for team selection
    mens_subset = pd.DataFrame(mens_rankings[mens_rankings[age_group_col].isin(age_groups)])
    mens_subset2 = wc_grouper(mens_subset, weight_class_colname, True) 
    mens_subset_final = org_col_adder(mens_subset2, org_cols)
    mens_exp_path = export_path + "/" + "Team_Alberta_Mens_" + age_groups[0] + ".csv"
    if export == True:
        mens_subset_final.to_csv(mens_exp_path, index=False)
    #Organizing woman for team selection 
    womans_subset = pd.DataFrame(womans_rankings[womans_rankings[age_group_col].isin(age_groups)])
    womans_subset2 = wc_grouper(womans_subset, weight_class_colname, True)  
    womans_subset_final = org_col_adder(womans_subset2, org_cols)
    female_exp_path = export_path + "/" + "Team_Alberta_Woman_" + age_groups[0] + ".csv"
    if export == True:
        womans_subset_final.to_csv(female_exp_path, index=False)
    return mens_subset_final, womans_subset_final



In [9]:
#Set folder dirrectory 
curr_dir = "/Users/milsbeary/Desktop/Weighlifting/Data/Rankings_Folder_1"

#See what is in the current dirrectory to make sure it is correct
print('All files')
direct_items = os.listdir(curr_dir)
for item in direct_items:
    print(item)


All files
Update_Sheet.csv
Master_Sheet_template.csv
.DS_Store
Masters_men_age_coefficents.csv
classification_tables_totals.xlsx
Masters_woman_age_coefficents.csv
Sinclair_Constants.csv


In [10]:
#Set up classification tables
mens_index_1, womans_index_1 = classification_setup(curr_dir, 'classification_tables_totals.xlsx')

#View to make sure we are kosher
print(mens_index_1)
print(womans_index_1)

#Setting up Sinclair constant table
sinclair_consts_1 = sinclair_constant_setup(curr_dir, 'Sinclair_Constants.csv')
#print(sinclair_consts_1)

#Setting up age constant classification tables
men_age_consts = age_const_setup(curr_dir, 'Masters_men_age_coefficents.csv')
#print(men_age_consts)
woman_age_consts = age_const_setup(curr_dir, 'Masters_woman_age_coefficents.csv')
#print(woman_age_consts)


  Category  Novice  Jr. Nats   Prov. 5  Prov. 4  Prov. 3  Prov. 2  Prov. 1  \
0       55     122      129.0    137.0    151.0    164.0    178.0    192.0   
1       61     132      140.0    148.0    163.0    178.0    192.0    207.0   
2       67     141      149.0    158.0    174.0    190.0    205.0    221.0   
3       73     149      157.0    167.0    184.0    200.0    217.0    234.0   
4       81     159      167.0    178.0    195.0    213.0    231.0    248.0   
5       89     167      176.0    186.0    205.0    224.0    242.0    261.0   
6       96     173      182.0    193.0    212.0    232.0    251.0    270.0   
7      102     177      187.0    198.0    218.0    238.0    257.0    277.0   
8      109     182      191.0    203.0    223.0    244.0    264.0    284.0   
9     109+     191      201.0    213.0    234.0    256.0    277.0    298.0   

   Sr. Nats  Int. 1  Int. 2  Next Gen. Elite  Elite Marker  
0     203.0   217.0   225.0            233.0  242.0    274  
1     220.0   234.0

In [19]:
#Builds a master sheet from the raw update sheet where all of the missing columns are filled in

#Set settings so we can view full dataframes 
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

#Building the mastersheet
updated_sheet = master_sheet_updater(curr_dir, 'Master_Sheet.csv', curr_dir, 'Update_Sheet.csv', mens_index_1, womans_index_1, sinclair_consts_1, men_age_consts, woman_age_consts)
print(updated_sheet)


             Last_Name   First_Name Sex Weight_Class      BW   Club   YOB  \
0               Severo        Jaxon   M          102  101.50    FSA  2005   
1              Lorenzo         Dave   M           73   73.00    OPF  1998   
2              Taborda     Jhonatan   M           73   73.00    OPF  2002   
3             McIntyre      Brandon   M           81   76.79    GGW  2004   
4               Sandee       Adrian   M           81   81.00    OPF  1989   
5               Truong       Dennis   M           81   80.00    TOP  2001   
6                Korte      Michael   M         109+  136.00    PPL  1990   
7              Lacomel       Darryn   M           61   61.00    OPF  1995   
8                 Lieu         Evan   M           67   65.00    MSC  1997   
9                 Park    Nathaniel   M           67   66.62    TOP  2004   
10              Belair        Dylan   M          109  106.00    UNA  1992   
11             Blakely      Zachary   M           89   85.00    OPF  1999   

/var/folders/bq/k35nlsld0q587dprw75c21bm0000gn/T/ipykernel_97677/209616367.py:213: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_master_frame = pd.concat([mod_update_frame, old_master_frame], axis=0).reset_index(drop=True)


In [23]:
#Building the male and female rankings

#Specifying columns to include 
oth_inc_cols = ["Last_Name", "First_Name", "Weight_Class", "BW", "Club", "YOB", "Date", "Total", "Class", "Competition_Group_Current_Age"]    

#Building the rankings
all_male_rankings, all_female_rankings = all_ranker(updated_sheet, '2024-06-01', '2025-05-31', 'First_Name', 'Last_Name', 'Weight_Class', 'BW_Sinclair', oth_inc_cols, curr_dir, True) 
print(all_male_rankings)
print(all_female_rankings)


    Rank          Last_Name   First_Name Weight_Class      BW Club   YOB  \
0      1               Chui     Jonathan           73   72.84  MSC  1998   
1      2             Severo        Jaxon          102   102.0  FSA  2005   
2      3  Cathcart-McKinnon        Aaron          102   101.9   MB  1997   
3      4             Cachia       Damien           73    71.7  UNA  1994   
4      5                 Lo        Derek           96    95.5  UNA  1995   
5      6            Klassen       Martin         109+  131.55  OPF  2002   
6      7             Pretty       Andrew           96    96.0  GGW  1992   
7      8               Luck     Mitchell          102  100.71  UNA  1996   
8      9             Browne       Jordan           89    89.0   MB  1997   
9     10            Lorenzo         Dave           73    73.0  OPF  1998   
10    11          Melemenis        Chris           81    80.6  VAL  1989   
11    12           Urquhart        Colin           89   88.97  FRP  1994   
12    13    

In [25]:
#Breaks up rankings into groups for a more detailed view
#Make sure all columns included here were included in the cell above

# <= Sr Male 
all_male_ranking_old = rankings_seperator(all_male_rankings, "Competition_Group_Current_Age", ["Sr", "Mstr"])
print(all_male_ranking_old)
#simple_exporter(all_male_ranking_old, curr_dir, "Mens_sr_rankings")

# <= Jr Male 
all_male_ranking_young = rankings_seperator(all_male_rankings, "Competition_Group_Current_Age", ["Jr", "Yth"])
print(all_male_ranking_young)
#simple_exporter(all_male_ranking_young, curr_dir, "Mens_jr_rankings")

# >= Sr Woman
all_fem_ranking_old = rankings_seperator(all_female_rankings, "Competition_Group_Current_Age", ["Sr", "Mstr"])
print(all_male_ranking_old)
#simple_exporter(all_fem_ranking_old, curr_dir, "Womans_sr_rankings")

# <= Jr Woman 
all_fem_ranking_young = rankings_seperator(all_female_rankings, "Competition_Group_Current_Age", ["Jr", "Yth"])
print(all_fem_ranking_young)
#simple_exporter(all_fem_ranking_young, curr_dir, "Womans_jr_rankings")


    Rank          Last_Name   First_Name Weight_Class      BW Club   YOB  \
0      1               Chui     Jonathan           73   72.84  MSC  1998   
1      2  Cathcart-McKinnon        Aaron          102   101.9   MB  1997   
2      3             Cachia       Damien           73    71.7  UNA  1994   
3      4                 Lo        Derek           96    95.5  UNA  1995   
4      5            Klassen       Martin         109+  131.55  OPF  2002   
5      6             Pretty       Andrew           96    96.0  GGW  1992   
6      7               Luck     Mitchell          102  100.71  UNA  1996   
7      8             Browne       Jordan           89    89.0   MB  1997   
8      9            Lorenzo         Dave           73    73.0  OPF  1998   
9     10          Melemenis        Chris           81    80.6  VAL  1989   
10    11           Urquhart        Colin           89   88.97  FRP  1994   
11    12               Kent        Miles           81    79.7  GGW  1997   
12    13    

In [22]:
#Builds dataframes for internal team selections 

#Sets columns needed for manual organization
the_org_cols = ["Accepted", "All forms"]

#Inclides columns we want
oth_inc_cols = ["Last_Name", "First_Name", "Weight_Class", "BW", "Club", "YOB", "Competition_Group_Current_Age" ,"Date", "Total"]    

#Selects sr teams
sr_male_team, sr_female_team = team_selector_internal(updated_sheet, '2024-06-01', '2025-05-31', 'First_Name', 'Last_Name', 'BW_Sinclair',["Sr", "Mstr"], "Competition_Group_Current_Age" , "Weight_Class", oth_inc_cols, the_org_cols, curr_dir, False)
print(sr_male_team)
print(sr_female_team)

#Selects jr teams
jr_male_team, jr_womans_team = team_selector_internal(updated_sheet, '2024-06-01', '2025-05-31', 'First_Name', 'Last_Name', 'BW_Sinclair',["Jr", "Yth"], "Competition_Group_Current_Age", "Weight_Class", oth_inc_cols, the_org_cols, curr_dir, False)
print(jr_male_team)
print(jr_womans_team)



    Rank          Last_Name   First_Name Weight_Class      BW Club   YOB  \
0      1               Chui     Jonathan           73   72.84  MSC  1998   
1      2             Cachia       Damien           73    71.7  UNA  1994   
2      3            Lorenzo         Dave           73    73.0  OPF  1998   
3      4            Taborda     Jhonatan           73    73.0  OPF  2002   
4      5             Flores      Russell           73   72.16  UNA  1985   
5      6              Perez      Patrick           73   70.89  VAL  2003   
6      7             Comeau         Jack           73   71.33  VAL  1981   
7      1  Cathcart-McKinnon        Aaron          102   101.9   MB  1997   
8      2               Luck     Mitchell          102  100.71  UNA  1996   
9      3                 Xu        Robin          102  101.02   AW  1988   
10     4              Kunik        Aaron          102    99.5  UNA  1998   
11     5               Berg       Johann          102   98.01  SYN  1994   
12     6    